In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import files
uploaded = files.upload()

Saving news.article.json to news.article.json


In [4]:
import io
df = pd.read_json(io.BytesIO(uploaded['news.article.json']))

In [5]:
df

,articleBody,dateModified,scrapedDate,source,title
0,"Sanjay Raut, a member of the Shiv Sena (UBT) p...",{'$date': '2023-10-25T06:35:50.000Z'},{'$date': '2023-10-27T13:12:18.339Z'},https://www.thehansindia.com/,Shiv Sena MP Sanjay Raut Responds To 'Hamas' R...
1,"Kozhikode (Kerala) [India], October 27 (ANI): ...",NaN,{'$date': '2023-10-27T13:12:45.595Z'},https://www.aninews.in/,At IUML's pro-Palestine rally in Kerala Tharoo...
2,"Mumbai, Oct 24 (PTI) Maharashtra Chief Ministe...",{'$date': '2023-10-25T02:14:27.000Z'},{'$date': '2023-10-27T13:12:18.339Z'},https://thefederal.com/,Uddhav buried Bal Thackeray's 'Hindutva' for p...
3,"Sensex, Nifty rebound over 1 pc after six sess...",NaN,{'$date': '2023-10-27T13:12:41.618Z'},https://english.varthabharati.in/,"New Bills replacing IPC, CrPC, Evidence Act wi..."
4,"October 26, 2023 08:15 pm | Updated 08:38 pm I...",{'$date': '2023-10-26T14:45:24.000Z'},{'$date': '2023-10-27T13:12:45.595Z'},https://www.thehindu.com/,"Israel biggest terrorist nation in the world, ..."
...,...,...,...,...,...
37416,Lebanese media are reporting renewed IDF strik...,NaN,{'$date': '2024-03-31T03:10:17.646Z'},https://www.timesofisrael.com/,Lebanese media reports renewed IDF strikes in ...
37417,Amid escalating tensions and concerns over pot...,{'$date': '2024-03-31T03:03:52.000Z'},{'$date': '2024-03-31T03:09:38.331Z'},https://www.deccanchronicle.com/,"US approves additional bombs, warplanes sales ..."
37418,"Haaretz.com, the online English edition of Haa...",{'$date': '2024-03-28T00:00:00.000Z'},{'$date': '2024-03-31T03:09:46.683Z'},https://www.haaretz.com/,Israel Publishes Video of Islamic Jihad Terror...
37419,"The UN secretary general, António Guterres, ha...",{'$date': '2024-03-31T00:00:00.000Z'},{'$date': '2024-03-31T03:09:50.586Z'},https://www.theguardian.com/,United Nations secretary general condemns expl...


In [6]:
columns_to_drop = ['scrapedDate', 'source', 'dateModified']
df.drop(columns=columns_to_drop, inplace=True)

In [7]:
contexts = df['articleBody']

In [8]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load pre-trained model and tokenizer
model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
def answer_question(question, context):
    # Tokenize question and context
    inputs = tokenizer(question, context, return_tensors="pt", padding=True, truncation=True)
    # Get answer from model
    with torch.no_grad():
        outputs = model(**inputs)
    answer_start_scores = outputs.start_logits
    answer_end_scores = outputs.end_logits
    answer_start = torch.argmax(answer_start_scores)
    answer_end = torch.argmax(answer_end_scores) + 1
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer

In [10]:
question = "What caused the Israel-Hamas war?"

all_contexts = " ".join(contexts.tolist())

answer = answer_question(question, all_contexts)
print("Question:", question)
print("Answer:", answer)

Question: What caused the Israel-Hamas war?
Answer: [CLS] what caused the israel - hamas war? [SEP]


In [ ]:
#Approach:-
#1. The first step is to choose a model for question-answering:
#I have chosen the suitable Bert based pre trained model (bert-large-uncased-whole-word-masking-finetuned-squad).
#The reason for choosing the model is it handling large size of data and can uncover semantic relationship between the
#text data and one more reason is that it take both upper case and lower case letters same.


In [ ]:
#2.	Load the data into the google colab which is in json format containing 37420 articles and have 5 columns.
#I worked only on articleBody because it contain the complete information and can help to answer the question in effective manner.

In [ ]:
#3.	The question and the context are tokenized using the BERT tokenizer.
#Tokenization is basically the conversion of text into tokens (words or subwords) and converting them into
#numerical representations suitable for input to the model.

In [ ]:
#4.	This function, answer_question, takes a question and a context as input.
#It first tokenizes the question and the context using the BERT tokenizer, converting them into a format suitable for the model.
#Then, it passes these tokenized inputs to the pre-trained BERT model. After obtaining the model's outputs, it identifies the start and
#end positions of the answer span within the context based on the highest logits (scores). Finally, it converts these token IDs back into
#a string representation to represents the predicted answer to the question.


In [ ]:
#5.	Then combines all context strings into one, then uses a function to find and print the answer to the question within the combined context.

In [11]:
!pip freeze > requirements.txt